# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察填補缺值以及 標準化 / 最小最大化 對數值的影響

# [作業重點]
- 觀察替換不同補缺方式, 對於特徵的影響 (In[4]~In[6], Out[4]~Out[6])
- 觀察替換不同特徵縮放方式, 對於特徵的影響 (In[7]~In[8], Out[7]~Out[8])

In [67]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
import time

data_path = '../Part02/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [68]:
[*zip(df.dtypes, df.columns)]

[(dtype('int64'), 'Pclass'),
 (dtype('O'), 'Name'),
 (dtype('O'), 'Sex'),
 (dtype('float64'), 'Age'),
 (dtype('int64'), 'SibSp'),
 (dtype('int64'), 'Parch'),
 (dtype('O'), 'Ticket'),
 (dtype('float64'), 'Fare'),
 (dtype('O'), 'Cabin'),
 (dtype('O'), 'Embarked')]

In [69]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

5 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']



In [70]:
# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
train_num = train_Y.shape[0]
df.head()

,Pclass,Age,SibSp,Parch,Fare
0,3,22.0,1,0,7.2500
1,1,38.0,1,0,71.2833
2,3,26.0,0,0,7.9250
3,1,35.0,1,0,53.1000
4,3,35.0,0,0,8.0500


# 作業1
* 試著在補空值區塊, 替換並執行兩種以上填補的缺值, 看看何者比較好?

In [71]:
df.mean()

Pclass     2.294882
Age       29.881138
SibSp      0.498854
Parch      0.385027
Fare      33.295479
dtype: float64

In [72]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 5 columns):
Pclass    1309 non-null int64
Age       1046 non-null float64
SibSp     1309 non-null int64
Parch     1309 non-null int64
Fare      1308 non-null float64
dtypes: float64(2), int64(3)
memory usage: 61.4 KB


In [73]:
# 空值補 -1, 做羅吉斯迴歸
df_m1 = df.fillna(-1)
train_X = df_m1[:train_num]
estimator = LogisticRegression()
score1 = cross_val_score(estimator, train_X, train_Y, cv=5).mean()
print('fillna_-1', score1)
"""
Your Code Here
"""
df_m2 = df.fillna(df.mean())
train_X = df_m2[:train_num]
estimator = LogisticRegression()
score2 = cross_val_score(estimator, train_X, train_Y, cv=5).mean()
print('fillna_mean:', score2)

df_m3 = df.fillna(df.median())
train_X = df_m3[:train_num]
estimator = LogisticRegression()
score3 = cross_val_score(estimator, train_X, train_Y, cv=5).mean()
print('fillna_median:',score3)

fillna_-1 0.6960299128976762
fillna_mean: 0.6981761033723469
fillna_median: 0.6992934218081011


C:\Users\Karma\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Karma\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Karma\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Karma\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Karma\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence t

使用-1, mean, median填補缺值後進行羅吉斯回歸，以median填補缺值之結果最好，故後續使用median填補缺值

# 作業2
* 使用不同的標準化方式 ( 原值 / 最小最大化 / 標準化 )，搭配羅吉斯迴歸模型，何者效果最好?

In [84]:
"""
Your Code Here
"""
a = time.time()
df_m3 = df.fillna(df.median())
train_X = df_m3[:train_num]
estimator = LogisticRegression()
score3 = cross_val_score(estimator, train_X, train_Y, cv=5).mean()
print('fillna_median_original:',score3)
b = time.time()
print(b-a)

a = time.time()
min_max = (df_m3 - df_m3.min())/(df_m3.max()-df_m3.min())
train_X = min_max[:train_num]
score_minmax = cross_val_score(LogisticRegression(), train_X, train_Y, cv=5).mean()
print('minmax:', score_minmax)
b = time.time()
print(b-a)

a = time.time()
stand = (df_m3 - df_m3.mean())/df_m3.std()
train_X = stand[:train_num]
score_stand = cross_val_score(LogisticRegression(), train_X, train_Y, cv=5).mean()
print('stand:', score_stand)
b = time.time()
print(b-a)

fillna_median_original: 0.6992934218081011
0.07595205307006836
minmax: 0.6982328807104934
0.04097938537597656
stand: 0.6970587140092496
0.03699755668640137


C:\Users\Karma\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Karma\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Karma\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Karma\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Karma\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence t

分別使用(原值 / 最小最大化 / 標準化)進行標準化，搭配羅吉斯迴歸模型，發現原值結果最好 (原值 > 最小最大化 > 標準化)，但經過標準化過後的資料執行時間較短